In [1]:
import numpy as np
import networkx as nx
import os
import pandas as pd
import random
import time
import json

In [2]:
datadir = '/data/zilu/cnt/'
# I put all bundle folders into one directory 
# Part1, Part2, Part3, Part4, Part5, and Part6, Part 7
# notice the part 1-5 are for training and validation (Ord)
# part 6 is only for validation [1164, 1179] (Large)
# part 7 is only for validation [1180, 1182] (Extra)

In [3]:
allfiles = os.listdir(datadir)
allfiles.remove('.DS_Store')
print(len(allfiles))

1182


In [4]:
emptyfiles = []
for fn in allfiles:
    path = datadir + fn
    if len(os.listdir(path)) == 0:
        emptyfiles.append(fn)
print(len(emptyfiles))
print(emptyfiles)
# we shall remove all emtpy files

22
['579', '584', '585', '575', '459', '589', '211', '530', '571', '583', '582', '573', '587', '574', '198', '577', '576', '581', '572', '578', '588', '586']


### retrive availabel files

In [5]:
data = pd.read_csv('/data/zilu/hsgnn/data.csv', index_col=0) 
# I have pre-porcessed the original results.xlsx into data.csv for good format

In [6]:
data.head()

,File name,# of Tubes,# walls per CNT,# of Atoms,# of Bonds,Bond Ratio,CNT height,Initial Dia (Ang.),Initial Dia (nm),Cross-sectional area (nm^2),Strain at Break,1/2 Stress (GPa),1/2 Strain,Strength (GPa),Elastic Modulus (GPa)
File #,,,,,,,,,,,,,,,
1,001-2N2M2,2.0,1.0,160.0,236.0,1.475000,24.169428,2.71,0.271,0.665705,0.236,27.573717,0.117,45.465992,101.576473
2,002-2N3M3,2.0,1.0,240.0,354.0,1.475000,24.498940,4.07,0.407,0.972903,0.248,35.445715,0.123,53.955534,311.154075
3,003-2N4M4,2.0,1.0,320.0,472.0,1.475000,24.828452,5.42,0.542,1.335309,0.259,42.759140,0.129,60.186624,418.153307
4,004-2N5M5,2.0,1.0,400.0,590.0,1.475000,25.157964,6.78,0.678,1.758293,0.258,38.734551,0.128,56.084452,447.525699
5,005-3N2M2,3.0,1.0,240.0,356.0,1.483333,24.169428,2.71,0.271,0.998558,0.238,39.745454,0.118,60.503616,162.529792


In [7]:
data.tail()

,File name,# of Tubes,# walls per CNT,# of Atoms,# of Bonds,Bond Ratio,CNT height,Initial Dia (Ang.),Initial Dia (nm),Cross-sectional area (nm^2),Strain at Break,1/2 Stress (GPa),1/2 Strain,Strength (GPa),Elastic Modulus (GPa)
File #,,,,,,,,,,,,,,,
1178,1178-16TWCNT12_15_18_frac,16.0,3.0,28704.0,42696.0,1.487458,24.595121,24.41,2.441,100.003693,0.223776,66.414990,0.110889,95.454832,755.997724
1179,1179-Colomer_64DWCNT14_7,64.0,2.0,78080.0,117120.0,1.500000,41.705664,14.50,1.450,168.334074,0.287712,67.835992,0.142857,123.201399,268.928583
1180,1180-5_9nmexperimental_bundle,60.0,2.0,102960.0,154440.0,1.500000,12.597300,59.00,5.900,1858.490816,0.170400,28.548106,0.085100,46.122525,523.450000
1181,1181-60_degree_junc,1.0,1.0,9590.0,14374.0,1.498853,468.800000,10.96,1.096,1.711050,0.090000,34.148351,0.049500,60.306156,729.916191
1182,1182-90_degree_junc,1.0,1.0,9138.0,13700.0,1.499234,468.800000,10.96,1.096,1.711050,0.088600,30.599199,0.044200,55.370933,728.550546


In [8]:
total_bundles = len(data) 
print(total_bundles)

1182


In [53]:
nans, avails = [], []
for i in range(total_bundles):
    if pd.isnull(data.iloc[i]).any():
        nans.append(data['File name'].iloc[i])
    else:
        avails.append(i+1)

In [54]:
len(nans), len(avails)

(24, 1158)

In [55]:
print(set(nans).difference(emptyfiles))
# those two structures do not finish the simulations, just ignore them

{'909-3N10M10_DC1_2', '910-4N10M10_DC1_2_3'}


In [56]:
fileIndices = avails
idmapping = {row[0]: row[1]['File name'] for row in data.iterrows() if row[0] in fileIndices}
if not os.path.exists('/data/zilu/files/cntmapping'):
    with open('/data/zilu/files/cntmapping', 'w') as f:
        f.write(json.dumps(idmapping))

In [59]:
OrdIndex = np.array([i for i in fileIndices if i <= 1163])
LargeIndex = np.array([i for i in fileIndices if i>=1164 and i<=1179])
ExtraIndex = np.array([i for i in fileIndices if i>=1180])
np.save('/data/zilu/files/ord.npy', OrdIndex)
np.save('/data/zilu/files/lrg.npy', LargeIndex)
np.save('/data/zilu/files/ext.npy', ExtraIndex)

In [72]:
os.listdir('/data/zilu/cnt/001-2N2M2')

['CNT-rip.data',
 'Surface.data_edited',
 'CNT-rip2.data',
 'CNT-equib.data',
 'Surface.data',
 'Surface_w_cg1_and_cge.data',
 'CNT-equib2.data']

## rename the simulated strucutre into 'exp.data' for unified use

In [97]:
for cnt in os.listdir(datadir):
    if cnt == '.DS_Store' or cnt in nans:
        continue
    allfiles = os.listdir(os.path.join(datadir, cnt))
    if 'CNT-equib.data' in allfiles:
        shutil.copy(os.path.join(datadir+cnt, 'CNT-equib.data'), os.path.join(datadir+cnt, 'exp.data'))
    
    elif 'cnt-equib.data' in allfiles:
        shutil.copy(os.path.join(datadir+cnt, 'cnt-equib.data'), os.path.join(datadir+cnt, 'exp.data'))
    
    elif 'CNT.data' in allfiles:
        shutil.copy(os.path.join(datadir+cnt, 'CNT.data'), os.path.join(datadir+cnt, 'exp.data'))
    
    elif 'cnt.data' in allfiles:
        shutil.copy(os.path.join(datadir+cnt, 'cnt.data'), os.path.join(datadir+cnt, 'exp.data'))
        
    else: # Part 2 naming convention is different
        dataname = cnt[4:] + '.data'
        shutil.copy(os.path.join(datadir+cnt, dataname), os.path.join(datadir+cnt, 'exp.data'))
        

In [99]:
os.listdir('/data/zilu/cnt/001-2N2M2')

['CNT-rip.data',
 'Surface.data_edited',
 'CNT-rip2.data',
 'exp.data',
 'CNT-equib.data',
 'Surface.data',
 'Surface_w_cg1_and_cge.data',
 'CNT-equib2.data']